In [9]:
import torch
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True

model = torch.hub.load('huggingface/pytorch-transformers', 'modelForQuestionAnswering', 'bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-large-uncased-whole-word-masking-finetuned-squad')

Using cache found in /Users/davidzhang/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /Users/davidzhang/.cache/torch/hub/huggingface_pytorch-transformers_master


In [17]:
# The format is paragraph first and then question
# text_1 = "Jim Henson was a puppeteer"
# text_2 = "Who was Jim Henson?"
text_1 = "Novak Djokovic is the GOAT"
text_2 = "Who is the GOAT?"
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)
tokens = tokenizer.convert_ids_to_tokens(indexed_tokens)
# for token, id in zip(tokens, indexed_tokens):
#     print((token, id))
sep_idx = indexed_tokens.index(tokenizer.sep_token_id)
len_question, len_answer = sep_idx + 1, len(indexed_tokens) - (sep_idx + 1)
segment_ids = [0] * len_question + [1] * len_answer
segment_tensors = torch.tensor([segment_ids])
tokens_tensor = torch.tensor([indexed_tokens])

('[CLS]', 101)
('novak', 19580)
('dj', 6520)
('##oko', 16366)
('##vic', 7903)
('is', 2003)
('the', 1996)
('goat', 13555)
('[SEP]', 102)
('who', 2040)
('is', 2003)
('the', 1996)
('goat', 13555)
('?', 1029)
('[SEP]', 102)
torch.Size([1, 15])
torch.Size([1, 15])


In [19]:
with torch.no_grad():
    out = model(tokens_tensor, token_type_ids=segment_tensors)

answer = tokenizer.decode(indexed_tokens[torch.argmax(out.start_logits):torch.argmax(out.end_logits)+1])
print(answer)

novak djokovic
